### In this Kaggle competition I’ll use time-series forecasting to forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer.

### Specifically, I'll build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores. I'll practice my machine learning skills with an approachable training dataset of dates, store, and item information, promotions, and unit sales.

In [226]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [227]:
#import datasets
train=pd.read_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\train.csv")
test=pd.read_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\test.csv")
holidays_events=pd.read_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\holidays_events.csv")
oil=pd.read_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\oil.csv")
stores=pd.read_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\stores.csv")
transactions=pd.read_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\transactions.csv")

In [166]:
#create id for saving id. This i'll use for saving results
id=test['id']

In [167]:
#datasets analize
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [168]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [169]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [170]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [171]:
oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [172]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [173]:
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [174]:
#comine all datasets into a single dataset
#oil-->train,test
oil.rename(columns={'dcoilwtico' : 'oil_price'}, inplace=True)
train=pd.merge(train, oil, on='date', how='left')
test=pd.merge(test, oil, on='date', how='left')

In [175]:
#stores-->train, test
train=pd.merge(train, stores, on='store_nbr', how='left')
test=pd.merge(test, stores, on='store_nbr', how='left')

In [176]:
train=train.drop(columns=['id'])
test=test.drop(columns=['id'])

In [177]:
#national_holiday_events-->train,test
holidays_events=holidays_events.drop(['locale','locale_name'],axis=1)
holidays_events.rename(columns={'type' : 'holidays_events'}, inplace=True)

train=pd.merge(train,holidays_events,on='date',how='left')
test=pd.merge(test,holidays_events,on='date',how='left')

In [178]:
train['date'] = train['date'].str.replace('-','')

In [179]:
test['date'] = test['date'].str.replace('-','')

In [180]:
train

,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,holidays_events,description,transferred
0,20130101,1,AUTOMOTIVE,0.000,0,NaN,Quito,Pichincha,D,13,Holiday,Primer dia del ano,False
1,20130101,1,BABY CARE,0.000,0,NaN,Quito,Pichincha,D,13,Holiday,Primer dia del ano,False
2,20130101,1,BEAUTY,0.000,0,NaN,Quito,Pichincha,D,13,Holiday,Primer dia del ano,False
3,20130101,1,BEVERAGES,0.000,0,NaN,Quito,Pichincha,D,13,Holiday,Primer dia del ano,False
4,20130101,1,BOOKS,0.000,0,NaN,Quito,Pichincha,D,13,Holiday,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,20170815,9,POULTRY,438.133,0,47.57,Quito,Pichincha,B,6,Holiday,Fundacion de Riobamba,False
3054344,20170815,9,PREPARED FOODS,154.553,1,47.57,Quito,Pichincha,B,6,Holiday,Fundacion de Riobamba,False
3054345,20170815,9,PRODUCE,2419.729,148,47.57,Quito,Pichincha,B,6,Holiday,Fundacion de Riobamba,False
3054346,20170815,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Quito,Pichincha,B,6,Holiday,Fundacion de Riobamba,False


In [181]:
test

,date,store_nbr,family,onpromotion,oil_price,city,state,type,cluster,holidays_events,description,transferred
0,20170816,1,AUTOMOTIVE,0,46.80,Quito,Pichincha,D,13,NaN,NaN,NaN
1,20170816,1,BABY CARE,0,46.80,Quito,Pichincha,D,13,NaN,NaN,NaN
2,20170816,1,BEAUTY,2,46.80,Quito,Pichincha,D,13,NaN,NaN,NaN
3,20170816,1,BEVERAGES,20,46.80,Quito,Pichincha,D,13,NaN,NaN,NaN
4,20170816,1,BOOKS,0,46.80,Quito,Pichincha,D,13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
28507,20170831,9,POULTRY,1,47.26,Quito,Pichincha,B,6,NaN,NaN,NaN
28508,20170831,9,PREPARED FOODS,0,47.26,Quito,Pichincha,B,6,NaN,NaN,NaN
28509,20170831,9,PRODUCE,1,47.26,Quito,Pichincha,B,6,NaN,NaN,NaN
28510,20170831,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,Quito,Pichincha,B,6,NaN,NaN,NaN


In [182]:
train.isnull().sum()

date                     0
store_nbr                0
family                   0
sales                    0
onpromotion              0
oil_price           955152
city                     0
state                    0
type                     0
cluster                  0
holidays_events    2551824
description        2551824
transferred        2551824
dtype: int64

In [183]:
test.isnull().sum()

date                   0
store_nbr              0
family                 0
onpromotion            0
oil_price           7128
city                   0
state                  0
type                   0
cluster                0
holidays_events    26730
description        26730
transferred        26730
dtype: int64

In [184]:
#fill NaN in oil_price does not fluctuate
train['oil_price']=train['oil_price'].fillna(method='bfill')
test['oil_price']=test['oil_price'].fillna(method='bfill')

In [185]:
train.isnull().sum()

date                     0
store_nbr                0
family                   0
sales                    0
onpromotion              0
oil_price                0
city                     0
state                    0
type                     0
cluster                  0
holidays_events    2551824
description        2551824
transferred        2551824
dtype: int64

In [186]:
test.isnull().sum()

date                   0
store_nbr              0
family                 0
onpromotion            0
oil_price              0
city                   0
state                  0
type                   0
cluster                0
holidays_events    26730
description        26730
transferred        26730
dtype: int64

In [187]:
#features preprocessing
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
train['city']= label_encoder.fit_transform(train['city'])
test['city']= label_encoder.fit_transform(test['city'])
 
train['city'].unique()
test['city'].unique()

array([18,  2, 10, 19,  9, 21,  7, 16,  0,  8, 20,  4,  1, 17, 15, 11,  3,
       12, 13,  6, 14,  5])

In [188]:
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
train['date']= label_encoder.fit_transform(train['date'])
test['date']= label_encoder.fit_transform(test['date'])
 
train['date'].unique()
test['date'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [189]:
# Encode labels in column 'species'.
train['family']= label_encoder.fit_transform(train['family'])
test['family']= label_encoder.fit_transform(test['family'])
 
train['family'].unique()
test['family'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

In [190]:
# Encode labels in column 'species'.
train['type']= label_encoder.fit_transform(train['type'])
test['type']= label_encoder.fit_transform(test['type'])
 
train['type'].unique()
test['type'].unique()

array([3, 2, 1, 4, 0])

In [191]:
# Encode labels in column 'species'.
train['state']= label_encoder.fit_transform(train['state'])
test['state']= label_encoder.fit_transform(test['state'])
 
train['state'].unique()
test['state'].unique()

array([12,  3,  2,  7, 14,  1, 11, 15,  6, 13,  9,  0,  8,  4,  5, 10])

In [192]:
# Encode labels in column 'species'.
train['transferred']= label_encoder.fit_transform(train['transferred'])
test['transferred']= label_encoder.fit_transform(test['transferred'])
 
train['transferred'].unique()
test['transferred'].unique()

array([1, 0])

In [193]:
train=train.drop(columns=['holidays_events','description'])
test=test.drop(columns=['holidays_events','description'])

In [194]:
train

,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type,cluster,transferred
0,0,1,0,0.000,0,93.14,18,12,3,13,0
1,0,1,1,0.000,0,93.14,18,12,3,13,0
2,0,1,2,0.000,0,93.14,18,12,3,13,0
3,0,1,3,0.000,0,93.14,18,12,3,13,0
4,0,1,4,0.000,0,93.14,18,12,3,13,0
...,...,...,...,...,...,...,...,...,...,...,...
3054343,1683,9,28,438.133,0,47.57,18,12,1,6,0
3054344,1683,9,29,154.553,1,47.57,18,12,1,6,0
3054345,1683,9,30,2419.729,148,47.57,18,12,1,6,0
3054346,1683,9,31,121.000,8,47.57,18,12,1,6,0


In [195]:
test

,date,store_nbr,family,onpromotion,oil_price,city,state,type,cluster,transferred
0,0,1,0,0,46.80,18,12,3,13,1
1,0,1,1,0,46.80,18,12,3,13,1
2,0,1,2,2,46.80,18,12,3,13,1
3,0,1,3,20,46.80,18,12,3,13,1
4,0,1,4,0,46.80,18,12,3,13,1
...,...,...,...,...,...,...,...,...,...,...
28507,15,9,28,1,47.26,18,12,1,6,1
28508,15,9,29,0,47.26,18,12,1,6,1
28509,15,9,30,1,47.26,18,12,1,6,1
28510,15,9,31,9,47.26,18,12,1,6,1


In [196]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         int32  
 1   store_nbr    int64  
 2   family       int32  
 3   sales        float64
 4   onpromotion  int64  
 5   oil_price    float64
 6   city         int32  
 7   state        int32  
 8   type         int32  
 9   cluster      int64  
 10  transferred  int32  
dtypes: float64(2), int32(6), int64(3)
memory usage: 186.4 MB


In [197]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         28512 non-null  int32  
 1   store_nbr    28512 non-null  int64  
 2   family       28512 non-null  int32  
 3   onpromotion  28512 non-null  int64  
 4   oil_price    28512 non-null  float64
 5   city         28512 non-null  int32  
 6   state        28512 non-null  int32  
 7   type         28512 non-null  int32  
 8   cluster      28512 non-null  int64  
 9   transferred  28512 non-null  int32  
dtypes: float64(1), int32(6), int64(3)
memory usage: 1.5 MB


In [198]:
# set independent and dependent variable
X=train.drop(columns=['sales'])
y=train.sales
X_test=test

In [199]:
X_test

,date,store_nbr,family,onpromotion,oil_price,city,state,type,cluster,transferred
0,0,1,0,0,46.80,18,12,3,13,1
1,0,1,1,0,46.80,18,12,3,13,1
2,0,1,2,2,46.80,18,12,3,13,1
3,0,1,3,20,46.80,18,12,3,13,1
4,0,1,4,0,46.80,18,12,3,13,1
...,...,...,...,...,...,...,...,...,...,...
28507,15,9,28,1,47.26,18,12,1,6,1
28508,15,9,29,0,47.26,18,12,1,6,1
28509,15,9,30,1,47.26,18,12,1,6,1
28510,15,9,31,9,47.26,18,12,1,6,1


In [200]:
#train test split for train models
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [201]:
LR=LinearRegression()

In [202]:
lasso_model = Lasso(alpha=0.1, random_state=42)

In [203]:
XGBR=XGBRegressor()

In [204]:
RFR= RandomForestRegressor(
    n_estimators=100,       # Уменьшенное число деревьев
    max_depth=15,           # Ограниченная глубина деревьев
    max_features='sqrt',    # Корень из числа признаков
    min_samples_split=10,   # Минимум образцов для разбиения
    min_samples_leaf=5,     # Минимум образцов в листьях
    n_jobs=-1,              # Использование всех ядер процессора
    random_state=42         # Фиксируем результат
)

In [205]:
LR.fit(X_train, y_train)

LinearRegression()

In [206]:
def model_evaluation(model, X_train, y_train, model_name):
    y_pred = model.predict(X_train)

    MAE = metrics.mean_absolute_error(y_train, y_pred)
    MSE = metrics.mean_squared_error(y_train, y_pred)
    RMSE = np.sqrt(MSE)
    R2=metrics.r2_score(y_train, y_pred)
    
    return pd.DataFrame([MAE, MSE, RMSE, R2], index=['MAE', 'MSE', 'RMSE' ,'R2'], columns=[model_name])

model_evaluation(LR, X_train, y_train, 'LinearRegression')

,LinearRegression
MAE,428.304304
MSE,980843.259194
RMSE,990.375312
R2,0.204849


In [207]:
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.1, random_state=42)

In [208]:
def model_evaluation(model, X_train, y_train, model_name):
    y_pred = lasso_model.predict(X_train)

    MAE = metrics.mean_absolute_error(y_train, y_pred)
    MSE = metrics.mean_squared_error(y_train, y_pred)
    RMSE = np.sqrt(MSE)
    R2=metrics.r2_score(y_train, y_pred)
    
    return pd.DataFrame([MAE, MSE, RMSE, R2], index=['MAE', 'MSE', 'RMSE' ,'R2'], columns=[model_name])

model_evaluation(lasso_model, X_train, y_train, 'Lasso')

,Lasso
MAE,428.294762
MSE,980843.285967
RMSE,990.375326
R2,0.204849


In [209]:
RFR.fit(X_train, y_train)

RandomForestRegressor(max_depth=15, max_features='sqrt', min_samples_leaf=5,
                      min_samples_split=10, n_jobs=-1, random_state=42)

In [210]:
def model_evaluation(model, X_train, y_train, model_name):
    y_pred = model.predict(X_train)

    MAE = metrics.mean_absolute_error(y_train, y_pred)
    MSE = metrics.mean_squared_error(y_train, y_pred)
    RMSE = np.sqrt(MSE)
    R2=metrics.r2_score(y_train, y_pred)
    
    return pd.DataFrame([MAE, MSE, RMSE, R2], index=['MAE', 'MSE', 'RMSE' ,'R2'], columns=[model_name])

model_evaluation(RFR, X_train, y_train, 'RandomForestRegressor')

,RandomForestRegressor
MAE,130.767838
MSE,150770.249135
RMSE,388.291449
R2,0.877773


In [211]:
XGBR.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [212]:
def model_evaluation(model, X_train, y_train, model_name):
    y_pred = model.predict(X_train)

    MAE = metrics.mean_absolute_error(y_train, y_pred)
    MSE = metrics.mean_squared_error(y_train, y_pred)
    RMSE = np.sqrt(MSE)
    R2=metrics.r2_score(y_train, y_pred)
    
    return pd.DataFrame([MAE, MSE, RMSE, R2], index=['MAE', 'MSE', 'RMSE' ,'R2'], columns=[model_name])

model_evaluation(XGBR, X_train, y_train, 'RandomForestRegressor')

,RandomForestRegressor
MAE,129.016531
MSE,154510.870671
RMSE,393.078708
R2,0.874741


In [213]:
y_pred1 = LR.predict(X_test)
y_pred2 = lasso_model.predict(X_test)
y_pred3 = RFR.predict(X_test)
y_pred4=XGBR.predict(X_test)

In [214]:
y_pred3

array([ 42.30890569,  40.49460568, 817.7536521 , ..., 750.1569613 ,
       816.04308798,  43.44035925])

In [215]:
y_pred4

array([ -54.871494,  -56.864758, -138.60078 , ...,  310.76175 ,
       -422.39325 ,   65.59275 ], dtype=float32)

In [232]:
df = pd.DataFrame({
    'id': id,
    'sales': y_pred3
})
print(df)

            id        sales
0      3000888    42.308906
1      3000889    40.494606
2      3000890   817.753652
3      3000891  1947.090523
4      3000892   137.329851
...        ...          ...
28507  3029395   382.117960
28508  3029396   146.116317
28509  3029397   750.156961
28510  3029398   816.043088
28511  3029399    43.440359

[28512 rows x 2 columns]


In [233]:
df1 = pd.DataFrame({
    'id': id,
    'sales': y_pred4
})
print(df)

            id        sales
0      3000888    42.308906
1      3000889    40.494606
2      3000890   817.753652
3      3000891  1947.090523
4      3000892   137.329851
...        ...          ...
28507  3029395   382.117960
28508  3029396   146.116317
28509  3029397   750.156961
28510  3029398   816.043088
28511  3029399    43.440359

[28512 rows x 2 columns]


In [237]:
submission_doc = df.to_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\submission.csv", index=False)

In [238]:
submission_doc = df1.to_csv("C:\\Users\\User\\Desktop\\store-sales-time-series-forecasting\\submission1.csv", index=False)

#### I've used 4 models. The most appropriate models are XGBregressor(87%) and Random Forest Regressor(87%). This results used for competition 